<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Calculation Methods </span> </p

In [0]:
#!import "./Extensions"

# Discount and Cumulate

Calculation method called from the Import Scopes. Expected inputs are nominal values and monthly discounting rate arrays and the period type which controls the details of the formula used. The monthly discounting rates can correspond to an empty array, this is the indication that this method should perform only the cumulation of the nominal withouth discounting. This is achieve by setting the monthly discounting to 1. 

In [0]:
public static double[] ComputeDiscountAndCumulate(this double[] nominalValues, double[] monthlyDiscounting, PeriodType periodType) 
{ 
    if(nominalValues == null) return Enumerable.Empty<double>().ToArray();
    if(!monthlyDiscounting.Any()) monthlyDiscounting = new double[]{1d}; //Empty discounting array triggers Cumulation. 

    var ret = new double[nominalValues.Length];
    
    if(periodType == PeriodType.BeginningOfPeriod)
    {
        for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = nominalValues[i] + ret.GetValidElement(i + 1) * monthlyDiscounting.GetValidElement(i/12);
        return ret;
    }
    
    for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = ( nominalValues[i] + ret.GetValidElement(i + 1) ) * monthlyDiscounting.GetValidElement(i/12);
    
    return ret;
}

In [0]:
public static double[] ComputeDiscountAndCumulateWithCreditDefaultRisk(this double[] nominalValues, double[] monthlyDiscounting, double nonPerformanceRiskRate) //NonPerformanceRiskRate consider to be constant in time. Refinement would it be an array that takes as input tau/t.
{ 
    if(!monthlyDiscounting.Any()) monthlyDiscounting = new double[]{1d}; //Empty discounting array triggers Cumulation.
    return Enumerable.Range(0, nominalValues.Length)
                     .Select( t => Enumerable.Range(t, nominalValues.Length-t)
                                             .Select( tau => nominalValues[tau] * Math.Pow(monthlyDiscounting.GetValidElement(t/12), tau-t+1) * (Math.Exp(-nonPerformanceRiskRate*(tau-t)) - 1) )
                                             .Sum() )
                     .ToArray();
}

In [0]:
public static IDataCube<RawVariable> ComputeDiscountAndCumulate ( this IDataCube<RawVariable> nominalRawVariables, double[] yearlyDiscountRates, AmountType[] amountTypes ) 
{
    if(nominalRawVariables == null) return Enumerable.Empty<RawVariable>().ToDataCube();
    var periodTypeByAmountType = amountTypes.ToDictionary(x => x.SystemName, x => x.PeriodType);
       
    return nominalRawVariables.Select(rv => {
        var values = rv.Values.ToArray();
        var cdcf = new double[values.Length];
        periodTypeByAmountType.TryGetValue(rv.AmountType, out var period);

        if(period == PeriodType.BeginningOfPeriod)
        {
            for (var i = cdcf.Length - 1; i >= 0; i--)
                cdcf[i] = values[i] + cdcf.GetValidElement(i + 1) * yearlyDiscountRates.GetValidElement(i/12);
        }
        else
        { 
            for (var i = cdcf.Length - 1; i >= 0; i--)
                cdcf[i] = ( values[i] + cdcf.GetValidElement(i + 1) ) * yearlyDiscountRates.GetValidElement(i/12);
        }
        return rv with { Values = cdcf };
    })
    .ToDataCube();
}

# Import Configuration 

## Data Nodes

In [0]:
public GroupOfInsuranceContract ExtendGroupOfContract(GroupOfInsuranceContract gic, IDataRow datarow) => gic;

In [0]:
public GroupOfReinsuranceContract ExtendGroupOfContract(GroupOfReinsuranceContract gric, IDataRow datarow) => gric;

## Data Node Parameters

In [0]:
public SingleDataNodeParameter ExtendSingleDataNodeParameter(SingleDataNodeParameter singleDataNodeParameter, IDataRow datarow) => singleDataNodeParameter;

## Economic Basis Driver default

In [0]:
public static string GetDefaultEconomicBasisDriver(string valuationApproach, string liabilityType) {
    return (valuationApproach, liabilityType) switch {
        (ValuationApproaches.BBA, _) => EconomicBases.L,
        (ValuationApproaches.VFA, _) => EconomicBases.C,
        (ValuationApproaches.PAA, LiabilityTypes.LIC) => EconomicBases.C,
        _ => EconomicBases.N,
    };
}

## Interpolate

In [0]:
public static double[] Interpolate(this double[] cashflowValues, CashFlowPeriodicity periodicity, InterpolationMethod interpolationMethod)
{ 
    if (periodicity == CashFlowPeriodicity.Monthly)
        return cashflowValues;
    
    var frequency = periodicity switch {
        CashFlowPeriodicity.Yearly => 12,
        CashFlowPeriodicity.Quarterly => 4,
        _ => 1
        };
    
    return interpolationMethod switch {
        InterpolationMethod.Start        => cashflowValues.SelectMany(v => Enumerable.Range(0, frequency).Select(x => x == 0 ? v : default)).ToArray(),
        InterpolationMethod.Uniform or _ => cashflowValues.SelectMany(v => Enumerable.Range(0, frequency).Select( _ => v / (double)frequency)).ToArray()
    };

}

## Change Sign Rules

In [0]:
public static int GetSign(string importFormat, (string AocType, string AmountType, string EstimateType, bool IsReinsurance) variable, Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache) => 1;

## Adjust Values for Cashflows

In [0]:
public static double[] AdjustValues(this double[] values, ImportArgs args, DataNodeData dataNodeData, int? AccidentYear) => values;

## Extend Parsed Raw Variables

In [0]:
public static async Task ExtendParsedVariables (this IWorkspace workspace, Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache) {}

## EstimateType And AmountType

In [0]:
public static Dictionary<string, HashSet<string>> GetAmountTypesByEstimateType(Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache)
{
    return new Dictionary<string, HashSet<string>>(){
        {EstimateTypes.RA, new string[]{}.ToHashSet()},
        {EstimateTypes.C, new string[]{}.ToHashSet()},
        {EstimateTypes.L, new string[]{}.ToHashSet()},
        {EstimateTypes.LR, new string[]{}.ToHashSet()},
   };
}

In [0]:
public static (string, string) ParseAmountAndEstimateType (this IDataRow datarow, string format, 
    Dictionary<Type, Dictionary<string, string>> dimensionsWithExternalId, 
    Dictionary<string, EstimateType> estimateTypes, 
    Dictionary<string, AmountType> amountTypes) 
{
    return (datarow.Field<string>(nameof(RawVariable.AmountType)), datarow.Field<string>(nameof(RawVariable.EstimateType)));
}

## Non Attributable AmountTypes

In [0]:
public static HashSet<string> GetNonAttributableAmountTypes() => new string[]{AmountTypes.NE}.ToHashSet();

## Technical Margin EstimateType

In [0]:
public static HashSet<string> GetTechnicalMarginEstimateType()
{
    return new []{EstimateTypes.C, EstimateTypes.L, EstimateTypes.LR, }.ToHashSet();
}

## Import Actuals EstimateType

In [0]:
public static HashSet<string> GetImportActualEstimateType()
{
    return new []{EstimateTypes.A, EstimateTypes.AA, EstimateTypes.OA}.ToHashSet();
}

## Simple Value and Opening Importer

In [0]:
public static HashSet<string> GetAocTypeWithoutCsmSwitch() => new []{AocTypes.BOP, AocTypes.EOP, AocTypes.AM, AocTypes.EA}.ToHashSet();

# Helper variables used in import calculation

Variables defined here and used in the calculation:
- AocTypeWithNoPv : used in [Present Value](../Import/2ImportScope-PresentValue#present-value-1) to trigger no calculation for some AoC Type.
- ReferenceAocSteps : used in [Reference AoC Step](../Import/1ImportScope-Identities#reference-aoc-step) to over rule the standard defition of reference AoC Step. 

In [0]:
public static class ComputationHelper{
    public static HashSet<AocStep> AocStepWithNoPv = new AocStep[]{new AocStep(AocTypes.EA, Novelties.C), new AocStep(AocTypes.AM, Novelties.C), new AocStep(AocTypes.RCU, Novelties.I),}.ToHashSet();
    public static HashSet<AocStep> AocStepWithNoCsm = new AocStep[]{new AocStep(AocTypes.CF, Novelties.C),new AocStep(AocTypes.WO, Novelties.C)}.ToHashSet();
    public static HashSet<string> ReinsuranceAocType = new []{AocTypes.CRU, AocTypes.RCU}.ToHashSet();
    
    public static Dictionary<AocStep, IEnumerable<AocStep>> ReferenceAocSteps => new ()
    {
        {new AocStep(AocTypes.EA, Novelties.C),  new []{new AocStep(AocTypes.CF, Novelties.C)}},
        {new AocStep(AocTypes.AM, Novelties.C),  new []{new AocStep(AocTypes.CL, Novelties.C)}},
        {new AocStep(AocTypes.EOP, Novelties.C), new []{new AocStep(AocTypes.CL, Novelties.C)}}, 
    };    
    public static Dictionary<string,string> ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string> {{EstimateTypes.A, EstimateTypes.APA}, {EstimateTypes.BE, EstimateTypes.BEPA}};
} 

# Initialize the values given the Period

In [0]:
public static double[] SetProjectionValue(double value, int period = 0) => 
    period == 0 || Math.Abs(value) > Precision
        ? Enumerable.Repeat(0d, period + 1).Select((y, i) => i == period ? value : y).ToArray()
        : null; 